In [1]:
import pandas as pd
from datetime import datetime

files = ["job_1999101.json","job_1988758.json", "job_2004889.json"]

seen_ids = set()

for file in files:
    file_name = "/dccstor/srosent2/generative/appen/" + file
    data = pd.read_json(file_name, lines=True)

    for index, row in data.iterrows():
        info = row['results']['judgments'][0]
        # print(row['results'])
            
        seen_ids.add(info['unit_data']['question_id'])

print(len(seen_ids))
print(seen_ids)

177
{'-5433421993461538564', '-5433998283119941152', '8847547952419838314', '2102201014821122308', '-6034041228964870497', '-5202569336471135378', '-2664875860902576660', '-3679693601791677766', '-3659701405687069690', '-6616030944495513115', '6663437261326194572', '2188641802032423949', '8915991454441317457', '-1735627592159571198', '7184331590045541410', '8484713537476807677', '7195430905493029565', '8717587106725834760', '5530825811853949020', '3550133641087898941', '6736163632077597066', '8581686621694664468', '517482339643940574', '-1813693802046160487', '2489004752830971283', '-4914858845663405521', '-5665052126355765119', '7236114255689242893', '5628470506183287603', '1409967239649772482', '-1342291290704725198', '3484265451298926829', '-6546770237964612453', '4069632166541178964', '-5959288268937806056', '2577707056846677575', '726040690787559971', '-5957516079482453552', '-480929800409517276', '-8531090588741234782', '-464770442921653920', '751853662280595560', '-2313102680940

In [2]:
# load tydi json and compute stats on the questions and answers
import glob
import gzip
import json
import spacy

nlp = spacy.load('en_core_web_sm')

train_data_dir = "/dccstor/srosent2/primeqa/data/nq-lfqa-train/*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data

In [3]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [4]:
def get_appen_data(data, type, exclude_ids):
    count = 0
    for example in data:
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        
        example_id = example['example_id']

        if str(example_id) in exclude_ids:
            continue

        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        passage_sentences = ""
        for sentence in nlp(passage_text).sents:
            passage_sentences += sentence.text + "&nbsp;"

        count += 1
        if count >= 30:
            break

        print(str(example_id) + "\t" + question + "\t" + example['document_title'] + "\t" + example['document_url'] + "\t" + passage_sentences + "\t" + minimal_text)
           

In [5]:
get_appen_data(train_data, "la", seen_ids)

6213884894002781583	the eye is on the sparrow bible verse	His Eye Is on the Sparrow	https://en.wikipedia.org//w/index.php?title=His_Eye_Is_on_the_Sparrow&amp;oldid=822339685	The theme of the song is inspired by the words of David in the Psalms and Jesus in the Gospel of Matthew in the Bible : `` I will instruct thee and teach thee in the way which thou shalt go : I will guide thee with mine eye ( Psalm 32 : 8 ) .&nbsp;`` Look at the birds of the air ; they neither sow nor reap nor gather into barns , and yet your heavenly Father feeds them .&nbsp;Are you not of more value than they ? ''&nbsp;( Matthew 6 : 26 ) and `` Are not two sparrows sold for a farthing ?&nbsp;and one of them shall not fall on the ground without your Father .&nbsp;But the very hairs of your head are all numbered .&nbsp;Fear ye not therefore , ye are of more value than many sparrows ''&nbsp;( Matthew 10 : 29 -- 31 ) .&nbsp;	
8849568873685505619	who has sold more records than anyone in history	List of best-selling mu